In [3]:
from qiskit_optimization import QuadraticProgram

qubo = QuadraticProgram()
# 変数を作ります
qubo.binary_var('q0')
qubo.binary_var('q1')
qubo.binary_var('q2')
qubo.binary_var('q3')
# 
linear={'q0': 4, 'q1': 4, 'q2': 4, 'q3': 4}
quadratic={('q0', 'q1'): 4, ('q0', 'q2'): 4, ('q1', 'q2'): 8, ('q1', 'q3'): 2, ('q2', 'q3'): 2}
qubo.minimize(linear=linear,quadratic=quadratic)
print(qubo)

minimize 4*q0*q1 + 4*q0*q2 + 8*q1*q2 + 2*q1*q3 + 2*q2*q3 + 4*q0 + 4*q1 + 4*q2 + 4*q3 (4 variables, 0 constraints, '')


In [13]:
# from qiskit_aer import Aer
# from qiskit_aer import QuantumCircuit, execute
# from qiskit.aqua.operators import I,X,Y
# from qiskit.aqua.components.initial_states import Custom

# mixer = ((X^X^I^I)+(Y^Y^I^I))/2 + ((I^I^X^X) + (I^I^Y^Y))/2

# initial_state_circuit = QuantumCircuit(4)
# initial_state_circuit.h(0)
# initial_state_circuit.cx(0, 1)
# initial_state_circuit.x(0)
# initial_state_circuit.h(2)
# initial_state_circuit.cx(2, 3)
# initial_state_circuit.x(2)
# initial_state_vec = execute(initial_state_circuit, Aer, get_backend('statevector_simulator')).result().get_statevector()
# initial_state = Custom(4, state_vector=initial_state_vec)



from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector, SparsePauliOp
from qiskit_aer import AerSimulator

# ミキサー（Pauli演算子で定義）
mixer = SparsePauliOp.from_list([
    ("XXII", 0.5),
    ("YYII", 0.5),
    ("IIXX", 0.5),
    ("IIYY", 0.5)
])

# 初期状態回路
initial_state_circuit = QuantumCircuit(4)
initial_state_circuit.h(0)
initial_state_circuit.cx(0, 1)
initial_state_circuit.x(0)
initial_state_circuit.h(2)
initial_state_circuit.cx(2, 3)
initial_state_circuit.x(2)

# ✅ 状態ベクトルを保存するように指示
initial_state_circuit.save_statevector()

# シミュレータ実行
simulator = AerSimulator(method='statevector')
compiled_circuit = transpile(initial_state_circuit, simulator)
result = simulator.run(compiled_circuit).result()

# ✅ これでちゃんと取得できます！
initial_state_vec = result.get_statevector()

# 初期状態オブジェクト化
initial_state = Statevector(initial_state_vec)


In [24]:
# from qiskit_optimization.algorithms import MinimumEigenOptimizer
# from qiskit.aqua import QuantumInstance
# from qiskit.aqua.algoritums import QAOA

# quantum_instance = QuantumInstance(Aer.get_backend('statevector_simulator'))
# step = 1
# qaoa_mes = QAOA(quantum_instance=quantum_instance, p=step,initial_state=initial_state,mixer=mixer)
# qaoa = MinimumEigenOptimizer(qaoa_mes)
# qaoa_result =  qaoa.solve(qubo)
# print(qaoa_result)



from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector, SparsePauliOp
from qiskit_aer import AerSimulator
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms.minimum_eigensolvers import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
# from qiskit.primitives import StatevectorSampler

# ミキサー演算子
mixer = SparsePauliOp.from_list([
    ("XXII", 0.5),
    ("YYII", 0.5),
    ("IIXX", 0.5),
    ("IIYY", 0.5)
])

# 初期状態回路
initial_state_circuit = QuantumCircuit(4)
initial_state_circuit.h(0)
initial_state_circuit.cx(0, 1)
initial_state_circuit.x(0)
initial_state_circuit.h(2)
initial_state_circuit.cx(2, 3)
initial_state_circuit.x(2)

# QAOA の設定
sampler = Sampler()
# sampler = StatevectorSampler() 
optimizer = COBYLA()
qaoa_mes = QAOA(
    sampler=sampler,
    optimizer=optimizer,
    reps=1,
    # # ミキサーを使用する場合
    # initial_state=initial_state_circuit,
    # mixer=mixer
)

# QUBO を解く
qaoa = MinimumEigenOptimizer(qaoa_mes)
result = qaoa.solve(qubo)

# 結果を表示
print(result)


/tmp/ipykernel_21994/1154555368.py:41: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


fval=0.0, q0=0.0, q1=0.0, q2=0.0, q3=0.0, status=SUCCESS
